In [1]:
import altair as alt
import pandas as pd

In [59]:
df = pd.DataFrame({
    'x': ['A', 'B', 'C', 'D', 'E'],
    'y': [5, -10, 15, -7, 5]
})

groups = 'abcde'
dfs = []
for i, group in enumerate(groups):
    dfs.append(df.copy())
    dfs[i]['group'] = f'group {group}'

df = pd.concat(dfs) 

In [79]:
chart = (
    alt.Chart(df)
    .transform_window(
        acc_y='sum(y)',
        frame=[None, 0],
        groupby=['group'],
    )
    .transform_window(
        lead_x='lead(x)',
        groupby=['group'],
    )
    .transform_calculate(
        calc_lead_x='datum.lead_x == null ? datum.x : datum.lead_x',
    )
)

chart = chart.mark_rule().encode(
    x='x:O',
    x2='calc_lead_x:O',
    # x='x:O',
    y='acc_y:Q',
    column='group',
)

# .properties(
#     width=200,
#     height=150,
# ).facet(
#     facet='group',
#     columns=2,
# ).resolve_axis(
#     x='independent',
# )

jchart = alt.JupyterChart(chart)
jchart

JupyterChart(spec={'config': {'view': {'continuousWidth': 300, 'continuousHeight': 300}}, 'data': {'name': 'da…

In [56]:
chart.save('chart.html')

In [112]:
data = [
    {"label": "Begin", "amount": 4000},
    # {"label": "Jan", "amount": 1707},
    {"label": "Feb", "amount": -1425},
    # {"label": "Mar", "amount": -1030},
    # {"label": "Apr", "amount": 1812},
    {"label": "May", "amount": -1067},
    # {"label": "Jun", "amount": -1481},
    {"label": "Jul", "amount": 1228},
    # {"label": "Aug", "amount": 1176},
    # {"label": "Sep", "amount": 1146},
    # {"label": "Oct", "amount": 1205},
    # {"label": "Nov", "amount": -1388},
    # {"label": "Dec", "amount": 1492},
    {"label": "End", "amount": 0},
]
source = pd.DataFrame(data)

sources = []
for i, group in enumerate(groups):
    sources.append(source.copy())
    sources[i]['group'] = f'group {group}'

source = pd.concat(sources)

# Define frequently referenced fields
amount = alt.datum.amount
label = alt.datum.label
window_lead_label = alt.datum.window_lead_label
window_sum_amount = alt.datum.window_sum_amount

# Define frequently referenced/long expressions
calc_prev_sum = alt.expr.if_(label == "End", 0, window_sum_amount - amount)
calc_amount = alt.expr.if_(label == "End", window_sum_amount, amount)
calc_text_amount = (
    alt.expr.if_((label != "Begin") & (label != "End") & calc_amount > 0, "+", "")
    + calc_amount
)

# The "base_chart" defines the transform_window, transform_calculate, and X axis
base_chart = alt.Chart(source).encode(
    x=alt.X("label:O", axis=alt.Axis(title="Months", labelAngle=0), sort=None)
)

color_coding = (
    alt.when((label == "Begin") | (label == "End"))
    .then(alt.value("#878d96"))
    .when(calc_amount < 0)
    .then(alt.value("#24a148"))
    .otherwise(alt.value("#fa4d56"))
)

bar_size = 35

bar = base_chart.mark_bar(size=bar_size).encode(
    y=alt.Y("calc_prev_sum:Q", title="Amount"),
    y2=alt.Y2("window_sum_amount:Q"),
    # color=color_coding,
    color=alt.Color('group:N').legend(None),
)

# The "rule" chart is for the horizontal lines that connect the bars
rule = base_chart.mark_rule(xOffset=-bar_size / 2, x2Offset=bar_size / 2).encode(
    y="window_sum_amount:Q",
    x2="calc_lead",
)

# Add values as text
text_pos_values_top_of_bar = base_chart.mark_text(baseline="bottom", dy=-4).encode(
    text=alt.Text("calc_sum_inc:N"),
    y="calc_sum_inc:Q",
)
text_neg_values_bot_of_bar = base_chart.mark_text(baseline="top", dy=4).encode(
    text=alt.Text("calc_sum_dec:N"),
    y="calc_sum_dec:Q",
)
text_bar_values_mid_of_bar = base_chart.mark_text(baseline="middle", fontSize=bar_size / 3).encode(
    text=alt.Text("calc_text_amount:N"),
    y="calc_center:Q",
    color=alt.value("white"),
)

chart = alt.layer(
    bar,
    rule,
    text_pos_values_top_of_bar,
    text_neg_values_bot_of_bar,
    text_bar_values_mid_of_bar
).properties(
    width=alt.Step(bar_size + 20),
    height=150
).facet(
    facet="group",
    columns=2,
).transform_window(
    window_sum_amount="sum(amount)",
    window_lead_label="lead(label)",
    groupby=["group"],
).transform_calculate(
    calc_lead=alt.expr.if_((window_lead_label == None), label, window_lead_label),
    calc_prev_sum=calc_prev_sum,
    calc_amount=calc_amount,
    calc_text_amount=calc_text_amount,
    calc_center=(window_sum_amount + calc_prev_sum) / 2,
    calc_sum_dec=alt.expr.if_(window_sum_amount < calc_prev_sum, window_sum_amount, ""),
    calc_sum_inc=alt.expr.if_(window_sum_amount > calc_prev_sum, window_sum_amount, ""),
).resolve_scale(
    x="independent"
)

chart

alt.FacetChart(...)

In [102]:
print(chart.to_json())

{
  "$schema": "https://vega.github.io/schema/vega-lite/v5.20.1.json",
  "columns": 2,
  "config": {
    "view": {
      "continuousHeight": 300,
      "continuousWidth": 300
    }
  },
  "data": {
    "name": "data-929441ba987ca3cf0d2a1fe93cd5b567"
  },
  "datasets": {
    "data-929441ba987ca3cf0d2a1fe93cd5b567": [
      {
        "amount": 4000,
        "group": "group a",
        "label": "Begin"
      },
      {
        "amount": 1707,
        "group": "group a",
        "label": "Jan"
      },
      {
        "amount": -1425,
        "group": "group a",
        "label": "Feb"
      },
      {
        "amount": -1030,
        "group": "group a",
        "label": "Mar"
      },
      {
        "amount": 1812,
        "group": "group a",
        "label": "Apr"
      },
      {
        "amount": -1067,
        "group": "group a",
        "label": "May"
      },
      {
        "amount": -1481,
        "group": "group a",
        "label": "Jun"
      },
      {
        "amount": 1228,
